In [24]:
#!/usr/bin/env python
"""Script to compare several variant calling csv file 
in the frame of Performance testing of plant virus 
(followong COST-DIVAS action)."""

import argparse
import os
import pandas as pd
import statistics
import numpy as np
import matplotlib.pyplot as plt

def make_percentage(freq):
    """
    """
    if "%" in freq:
        freq = float(freq.split("%")[0])
        freq = freq/100
    return float(freq)
    

if __name__ == "__main__":

    #TODO argparse use argument ?
    #col_name = ["Lab", "Type", "Reference", \
    #    "Allele", "Average_quality","Personal validation", "Comment", "Input_file", \
    #    "Org_ref", "File_name", "Pos_1", "Pos_1", "Pos_2", "frequency_1", "frequency_2"]
    in_dir = "/mnt/c/Users/johan/OneDrive/Bureau/INEXVIR_johan/PT3/Result"
    # in_file = "PART_Cleanned_PT3.csv"
    #in_file = "Cleanned_PT3.csv"
    in_file = "Cleanned_PT3_1percent.csv"
   
    #exp_file = "Expected_result.csv"
    exp_file = "Expected_result_1percent.csv"

    #clean_virus_data = pd.DataFrame(columns=col_name)
    clean_virus_data = pd.DataFrame()
    
    input_path = os.path.join(in_dir, in_file)
    exp_path = os.path.join(in_dir, exp_file)
    
    clean_virus_data = pd.read_csv(input_path, sep=";", header=0)
    exp_virus_data = pd.read_csv(exp_path, sep=";", header=0)

    print("Job done")
    


Job done


In [25]:
print(len(clean_virus_data))

8828


In [26]:
result_data2 = pd.DataFrame(columns=["Lab", "Sample", "Case", "Cov_1", "Freq_1", "Org_ref", 
                                    "Type", "Pos", "Ref", "Allele", "Validation", "Comment"])
data_virus = clean_virus_data.copy(deep=True)
len_count=0
len_count2=0
list_uniq_ID = []


def make_result_df(case, list_el, result_data2):
    a_series  = pd.Series([list_el[0], list_el[1], case, list_el[3], list_el[4], list_el[5], 
        list_el[6], list_el[7], list_el[8], list_el[9], list_el[10], list_el[11]], index = result_data2.columns)
    result_data2 = result_data2.append(a_series, ignore_index=True)
    
    return result_data2
    
for index2, row2 in data_virus.iterrows():
    is_exp = False
    is_nuc = False
    is_pos = False
    is_type = False
    Correct_exp_validation_Yes = False
    Wrong_exp_validation_No = False
    Correct_nuc_validation_No = False
    Wrong_nuc_validation_Yes = False
    Correct_pos_validation_No = False
    Wrong_pos_validation_Yes = False
    Correct_type_validation_No = False
    Wrong_type_validation_Yes = False
    lab = row2["Lab"]
    Sample = row2["Input_file"]
    Cov_1 = row2["Cov_1"]
    Freq_1 = row2["frequency_1"]
    Org_ref = row2["Org_ref"]
    Type = row2["Type"]
    Pos = row2["Pos_1"]
    Ref = row2["Reference"]
    Allele = row2["Allele"]
    Validation = row2["Personal validation"]
    Comment = row2["Comment"]
    pos_list = [Pos, Pos+1, Pos-1]
    
    list_el = [lab, Sample, "", Cov_1, Freq_1, Org_ref, 
    Type, Pos, Ref, Allele, Validation, Comment]
    for index, row_exp in exp_virus_data.iterrows():        
        # if on the same sample (1,2 or 3)
        if str(row_exp["Input_file"]) == row2["Input_file"][0]:
            # use a limit of detection frequency
            #if row_exp["frequency_1"]>= 0.02:
                # if mutation type identified is the same
            if row_exp["Type"] == row2["Type"]:
                # if position are the same
                if row_exp["Pos_1"] in pos_list:
                    # if nucleotide change is the same                     
                    if row_exp["Reference"] == row2["Reference"] and row_exp["Allele"] == row2["Allele"]:
                        is_exp = True
                        # if pass personal validation
                        if row2["Personal validation"] == "Yes":
                            Correct_exp_validation_Yes = True

                            break
                        elif row2["Personal validation"] == "No":    
                            Wrong_exp_validation_No = True
                            break
                        else:
                            break
                    else: # ref or allele is different for same sample/type/Pos_1 (above Freq_limit)
                        is_nuc = True
                        #result_data2 = make_result_df("Wrong_nucleotide", list_el, result_data2)
                        if row2["Personal validation"] == "No": 
                            Correct_nuc_validation_No = True
                        elif row2["Personal validation"] == "Yes": 
                            Wrong_nuc_validation_Yes = True
                else: # Pos_1 is different for same sample/type (above Freq_limit)
                    is_pos = True
                    #result_data2 = make_result_df("Wrong_position", list_el, result_data)
                    if row2["Personal validation"] == "No": 
                        Correct_pos_validation_No = True
                    elif row2["Personal validation"] == "Yes": 
                        Wrong_pos_validation_Yes = True

            else: # Type is different for same sample
                is_type = True
                #result_data2 = make_result_df("Wrong_mutation_type", list_el, result_data2)
                if row2["Personal validation"] == "No": 
                    Correct_type_validation_No = True
                    break
                elif row2["Personal validation"] == "Yes": 
                    Wrong_type_validation_Yes = True

                    break
            #else: # Frequency is different for same sample/type
                #pass
        # if NOT on the same sample
    # end of exp_virus_data loop
    
    if is_exp:
        if Correct_exp_validation_Yes:
            result_data2 = make_result_df("exp_Correct_validation_Yes", list_el, result_data2)
        elif Wrong_exp_validation_No:                                 
            result_data2 = make_result_df("exp_Wrong_validation_No", list_el, result_data2)
        else:
            result_data2 = make_result_df("exp_validation_unknown", list_el, result_data2)
    else:
        if is_type:   
            if Correct_type_validation_No: 
                result_data2 = make_result_df("type_Correct_validation_No", list_el, result_data2)
            elif Wrong_type_validation_Yes: 
                result_data2 = make_result_df("type_Wrong_validation_Yes", list_el, result_data2)   
            else:
                result_data2 = make_result_df("type_validation_unknown", list_el, result_data2)
            continue
        if is_nuc:  
            if Correct_nuc_validation_No: 
                result_data2 = make_result_df("nuc_Correct_validation_No", list_el, result_data2)
            elif Wrong_nuc_validation_Yes: 
                result_data2 = make_result_df("nuc_Wrong_validation_Yes", list_el, result_data2)
            else:
                result_data2 = make_result_df("nuc_validation_unknown", list_el, result_data2)    
            continue
        if is_pos: 
            if Correct_pos_validation_No: 
                result_data2 = make_result_df("pos_Correct_validation_No", list_el, result_data2)
            elif Wrong_pos_validation_Yes: 
                result_data2 = make_result_df("pos_Wrong_validation_Yes", list_el, result_data2)
            else:
                result_data2 = make_result_df("pos_validation_unknown", list_el, result_data2)

        

# l_index_pos_allele == TPs
# l_index_pos_al_validationY == TPsc == TPmv
# l_index_pos_al_validationN == FNmv
print("All done")
print(len_count)
print(len_count2)
print(len(result_data2))



All done
0
0
8828


In [27]:
from pivottablejs import pivot_ui

pivot_ui(result_data2)

In [7]:
for index, row in result_data2.iterrows():
    Lab = row["Lab"]
    if Lab == "H":
        if row["Sample"] == "2B":
            if row["Case"] == "exp_Correct_validation_Yes":
                pass
                #print(row)
print('pass')

pass


In [4]:
# 0.6	0.2	0.126	0.1	0.06	0.02	0.01

list_s1_pos_60p = [339, 900, 901, 902, 903, 904, 905, 906, 6336, 6337] #3
list_s2_pos_60p = [861] #1
list_s1_pos_20p = [2668, 319, 2191] #3
list_s2_pos_20p = [4918] #1
list_s3_pos_20p = [1983, 4031, 5067] #3 
list_s3_pos_12p = [217, 2752] #2
list_s1_pos_10p = [3830, 5439, 5440, 5894] #3
list_s2_pos_10p = [339] #1
list_s1_pos_06p = [2170, 3769, 3912] #3
list_s2_pos_06p = [3830] #1
list_s3_pos_06p = [5489, 4768, 3021, 3022, 3023, 3024, 925, 1646, 3390, 3391 ,3392, 3393] #6
list_s1_pos_02p = [2672, 4090] #2
list_s2_pos_02p = [2576] # 1
list_s1_pos_01p = [3371, 6380, 6381, 6382, 2557] #3
list_s2_pos_01p = [2916] #1
list_s3_pos_01p = [1258, 1599, 1600, 1601, 1602, 1603, 2363, 2364, 2365, 2366] #3
result_60p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_20p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_12p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_10p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_06p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_02p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])
result_01p = pd.DataFrame(columns=["Lab", "Sample", "Pos", "Validation", "Freq", "Case", "Comment"])

count = 0

list_sample = ["1BE", "1BF", "1BG", "1CE", "1CF", "1CG", "1DE", "1DF", "1DG", "2BE", "2BF", "2BG", 
               "2CE", "2CF", "2CG", "2DE", "2DF", "2DG", "3BE", "3BF", "3BG", "3CE", "3CF", "3CG", 
               "3DE", "3DF", "3DG"]
list_case = ["exp_Correct_validation_Yes", "exp_Wrong_validation_No", "exp_validation_unknown"] #TPs
for index, row in result_data2.iterrows():
    case = row["Case"]
    if case in list_case:
        freq = row["Freq_1"]
        position = row["Pos"]
        lab = row["Lab"]
        sample = row["Sample"]
        valid = row["Validation"]
        comment = row["Comment"]
        if sample in list_sample:
            if sample.startswith("1"):
                for el in list_s1_pos_60p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_60p.columns)
                        result_60p = result_60p.append(a_series, ignore_index=True)
                for el in list_s1_pos_20p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_20p.columns)
                        result_20p = result_20p.append(a_series, ignore_index=True)                   
                for el in list_s1_pos_10p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_10p.columns)
                        result_10p = result_10p.append(a_series, ignore_index=True) 
                for el in list_s1_pos_06p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_06p.columns)
                        result_06p = result_06p.append(a_series, ignore_index=True) 
                for el in list_s1_pos_02p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_02p.columns)
                        result_02p = result_02p.append(a_series, ignore_index=True) 
                for el in list_s1_pos_01p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_01p.columns)
                        result_01p = result_01p.append(a_series, ignore_index=True)                     
            if sample.startswith("2"):
                for el in list_s2_pos_60p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_60p.columns)
                        result_60p = result_60p.append(a_series, ignore_index=True)
                for el in list_s2_pos_20p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_20p.columns)
                        result_20p = result_20p.append(a_series, ignore_index=True)                   
                for el in list_s2_pos_10p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_10p.columns)
                        result_10p = result_10p.append(a_series, ignore_index=True) 
                for el in list_s2_pos_06p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_06p.columns)
                        result_06p = result_06p.append(a_series, ignore_index=True) 
                for el in list_s2_pos_02p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_02p.columns)
                        result_02p = result_02p.append(a_series, ignore_index=True) 
                for el in list_s2_pos_01p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_01p.columns)
                        result_01p = result_01p.append(a_series, ignore_index=True)
            if sample.startswith("3"):
                for el in list_s3_pos_20p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_20p.columns)
                        result_20p = result_20p.append(a_series, ignore_index=True) 
                for el in list_s3_pos_12p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_12p.columns)
                        result_12p = result_12p.append(a_series, ignore_index=True)
                for el in list_s3_pos_06p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_06p.columns)
                        result_06p = result_06p.append(a_series, ignore_index=True) 
                for el in list_s3_pos_01p:
                    if position == el or position == el+1 or position == el-1:
                        a_series  = pd.Series([lab,sample,position,valid,freq,case,comment], index = result_01p.columns)
                        result_01p = result_01p.append(a_series, ignore_index=True)        
                
print("freq analysis done")

freq analysis done


In [6]:
from pivottablejs import pivot_ui

pivot_ui(result_01p)